<!-- <div style="background-color: rgb(247, 230, 202); border: 4px solid rgb(162, 87, 79); border-radius: 40px; padding: 20px; font-family: 'Roboto'; color: rgb(162, 87, 79); text-align: left; font-size: 120%;">
    <ul style="list-style-type: square; padding-left: 20px;">
        <li style="margin-top: 10px;">HLA columns are recalculated as per <a href="https://www.kaggle.com/code/albansteff/cibmtr-eda-ensemble-model-recalculate-hla" style="color: #A2574F; text-decoration: underline;">this</a> notebook.</li>
        <li style="margin-top: 10px;">Missing values are replaced with:
            <ul style="list-style-type: circle; margin-top: 10px; margin-bottom: 10px;">
                <li>-1 for numeric columns</li>
                <li>Unknown for categorical columns</li>
            </ul>
        </li>
        <li style="margin-top: 10px;">
            LightGBM and CatBoost are trained on 3 different targets, estimated from the survival models:
            <ul style="list-style-type: circle; margin-top: 10px; margin-bottom: 10px;">
                <li>Cox</li>
                <li>Kaplan-Meier</li>
                <li>Nelson-Aalen</li>
            </ul>
        </li>
        <li style="margin-top: 10px;">Two additional CatBoost model are trained, with Cox loss function.</li>
        <li style="margin-top: 10px;">As per <a href="https://www.kaggle.com/competitions/equity-post-HCT-survival-predictions/discussion/553061" style="color: #A2574F; text-decoration: underline;">this</a> discussion post, the target is consisted of the Out-of-Fold predictions of the survival models on the validation folds to prevent target leakage.</li>
        <li style="margin-top: 10px;">
            The ensemble prediction for each sample is computed as:
            <ul style="list-style-type: circle; margin-top: 10px; margin-bottom: 10px;">
                <p style="margin-top: 10px; font-size: 110%; color: #A2574F; font-family: 'Roboto'; text-align: left;">
                    $ \text{preds}_{\text{ensemble}} = \sum_{i=1}^{n} w_i \cdot \text{rankdata}(\text{preds}_i) $
                </p>
                where $n$ is the number of models, $w_i$ is the weight assigned to the $i$-th model, and $\text{rankdata}(\text{preds}_i)$ is the rank of predictions from the $i$-th model.
            </ul>
        </li>
        <li style="margin-top: 10px;">Last but not least, since the competition metric evaluates only the order of predictions and not their magnitude, the model weights are not required to sum to 1, nor should the predictions fall within a predefined range.</li>
    </ul>
</div> -->

<p style="font-size: 150%; text-align: left; border-radius: 40px 40px; color: rgb(232, 95, 9); font-weight: bold;">参考にしたNotebook</p>  

リンク  
- アンサンブル：https://www.kaggle.com/code/andreasbis/cibmtr-eda-ensemble-model  
- NN：https://www.kaggle.com/code/dreamingtree/single-nn-with-pairwise-ranking-loss-0-689-lb

<p style="font-size: 120%; text-align: left; border-radius: 40px 40px; color: rgb(244, 62, 7); font-weight: bold;">
概要
</p>  

- 特徴量エンジニアリング
    - 特徴量作成
        - ``hla_high_xxx``や``hla_low_xxx``といった特徴量は自分で作り直す
- 欠損値補完
    - 数値列は``-1``
    - カテゴリ列は``Unknown``
- モデル
    - 種類（計８つのモデル）
        - ２つの予測モデル（LightGBMとCatBoost）×３つの生存確率推定モデル
            - コックス比例ハザードモデル
                - モデル概要
                    - セミパラメトリックモデル
                    - 共変量（複数の説明変数）を考慮できる
                    - 返す値は「相対的なリスクの大きさ」
                - データ処理
                    - １値しか取らないカラムを削除
                    - カテゴリ変数について``one hot encoding``を実施
            - カプラン・マイヤー推定量
                - ノンパラメトリックモデル（データからそのまま推定・特定の確率分布を仮定しない）
                - 共変量を考慮しない単純な推定
                - 生存確率を計算（０〜１）
                $$
                \begin{align*}
                S(t) &= \prod_{t_i \leq t} \left( 1 - \frac{d_i}{n_i} \right)  
                \end{align*}
                $$

                $$
                \begin{align*}
                t_i &\text{ はイベント発生時点} \\
                d_i &\text{ はその時点でのイベント発生個体数} \\
                n_i &\text{ はその時点での生存個体数}
                \end{align*}
                $$
            - ネルソン・アーレン推定量
                - ノンパラメトリックモデル（データからそのまま推定・特定の確率分布を仮定しない）
                - 共変量を考慮しない単純な推定
                - 累積ハザード確率を計算（０〜１）
                $$
                \begin{align*}
                H(t) &= \sum_{t_i \leq t} \frac{d_i}{n_i}
                \end{align*}
                $$
                $$
                \begin{align*}
                t_i &\text{ はイベント発生時点} \\
                d_i &\text{ はその時点でのイベント発生個体数} \\
                n_i &\text{ はその時点での生存個体数}
                \end{align*}
                $$
            - カプラン・マイヤーとネルソン・アーレンの違い
                - 「死亡のイベントが発生した時間に関数値が変化するという性質」は共通
                - ネルソン・アーレンは、「小さいサイズの標本に対してよりよい性質をもつ」
        - CatBoost（木の深さ優先 / 損失減少優先）× Cox回帰に基づく損失関数
            - CatBoost
                - ハイパーパラメータの設定の違い（木の深さ優先 / 損失減少優先）により2種類作成
                - 損失関数として「Cox回帰」を指定
            - Cox回帰に基づく損失関数
                - targetの設定
                    - 特定のイベントが発生せずに生存している期間を測定する指標を用いる
                    - イベントが発生しなかったデータについては生存時間を負にすることで、イベントが発生しなかったデータポイントを明確に区別できる
                - 予測モデル
                    - CatBoostにおいて、Cox回帰に基づく損失関数を使用
    - 特徴
        - ターゲットリークを防ぐため、ターゲットは検証フォールドにおけるサバイバルモデルのOut-of-Fold予測で構成される
            - https://www.kaggle.com/competitions/equity-post-HCT-survival-predictions/discussion/553061
        - アンサンブル予測は各サンプルについて以下のように計算される
            - アンサンブル予測 = Σ(wi * rankdata(predsi)) 
                - ここで、wiは i 番目のモデルに割り当てられた重み（<font color = "red">分析者が指定</font>）
                - rankdata(predsi)は i 番目のモデルの予測のランク
            - 競争メトリックが予測の大きさではなく順序のみを評価するため、モデルの重みは1に合計する必要はなく、予測も予め定義された範囲内である必要はありません。
- 精度評価
    - 層別C統計量を使用
        - 元のノートブックではmetricライブラリのscore関数を使用していたが、利用できないので代替
        - 層別C統計量は以前から使用していたもの


<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">ライブラリ</p>

In [ ]:
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_lightning-2.4.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/torchmetrics-1.5.2-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_tabnet-4.1.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/einops-0.7.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_tabular-1.1.1-py2.py3-none-any.whl

In [ ]:

from metric import score
import pandas as pd
import numpy as np
from warnings import filterwarnings
import joblib
from pathlib import Path
filterwarnings('ignore')

ROOT_DATA_PATH = Path(r"/kaggle/input/equity-post-HCT-survival-predictions")

pd.set_option('display.max_columns', 100)

train = pd.read_csv(ROOT_DATA_PATH.joinpath("train.csv"))
test = pd.read_csv(ROOT_DATA_PATH.joinpath("test.csv"))

CATEGORICAL_VARIABLES = [
    # Graft and HCT reasons
    'dri_score', 'graft_type', 'prod_type', 'prim_disease_hct',

    # Patient health status (risk factors)
    'psych_disturb', 'diabetes', 'arrhythmia', 'vent_hist', 'renal_issue', 'pulm_moderate',
    'pulm_severe', 'obesity', 'hepatic_mild', 'hepatic_severe', 'peptic_ulcer', 'rheum_issue',
    'cardiac', 'prior_tumor', 'mrd_hct', 'tbi_status', 'cyto_score', 'cyto_score_detail', 

    # Patient demographics
    'ethnicity', 'race_group',

    # Biological matching with donor
    'sex_match', 'donor_related', 'cmv_status', 'tce_imm_match', 'tce_match', 'tce_div_match',

    # Medication/operation related data
    'melphalan_dose', 'rituximab', 'gvhd_proph', 'in_vivo_tcd', 'conditioning_intensity'
]

HLA_COLUMNS = [
    'hla_match_a_low', 'hla_match_a_high',
    'hla_match_b_low', 'hla_match_b_high',
    'hla_match_c_low', 'hla_match_c_high',
    'hla_match_dqb1_low', 'hla_match_dqb1_high',
    'hla_match_drb1_low', 'hla_match_drb1_high',
    
    # Matching at HLA-A(low), -B(low), -DRB1(high)
    'hla_nmdp_6',
    # Matching at HLA-A,-B,-DRB1 (low or high)
    'hla_low_res_6', 'hla_high_res_6',
    # Matching at HLA-A, -B, -C, -DRB1 (low or high)
    'hla_low_res_8', 'hla_high_res_8',
    # Matching at HLA-A, -B, -C, -DRB1, -DQB1 (low or high)
    'hla_low_res_10', 'hla_high_res_10'
]

OTHER_NUMERICAL_VARIABLES = ['year_hct', 'donor_age', 'age_at_hct', 'comorbidity_score', 'karnofsky_score']
NUMERICAL_VARIABLES = HLA_COLUMNS + OTHER_NUMERICAL_VARIABLES

TARGET_VARIABLES = ['efs_time', 'efs']
ID_COLUMN = ["ID"]


def preprocess_data(df):
    df[CATEGORICAL_VARIABLES] = df[CATEGORICAL_VARIABLES].fillna("Unknown")
    df[OTHER_NUMERICAL_VARIABLES] = df[OTHER_NUMERICAL_VARIABLES].fillna(df[OTHER_NUMERICAL_VARIABLES].median())

    return df

train = preprocess_data(train)
test = preprocess_data(test)


def features_engineering(df):
    # Change year_hct to relative year from 2000
    df['year_hct'] = df['year_hct'] - 2000
    
    return df


train = features_engineering(train)
test = features_engineering(test)

train[CATEGORICAL_VARIABLES] = train[CATEGORICAL_VARIABLES].astype('category')
test[CATEGORICAL_VARIABLES] = test[CATEGORICAL_VARIABLES].astype('category')

FEATURES = train.drop(columns=['ID', 'efs', 'efs_time']).columns.tolist()

In [ ]:
from xgboost import XGBRegressor, XGBClassifier
import xgboost
print("Using XGBoost version",xgboost.__version__)
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

FOLDS = 5
kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

oof_xgb = np.zeros(len(train))
pred_efs = np.zeros(len(test))

for i, (train_index, test_index) in enumerate(kf.split(train, train["efs"])):

    print("#"*25)
    print(f"### Fold {i+1}")
    print("#"*25)
    
    x_train = train.loc[train_index, FEATURES].copy()
    y_train = train.loc[train_index, "efs"]
    x_valid = train.loc[test_index, FEATURES].copy()
    y_valid = train.loc[test_index, "efs"]
    x_test = test[FEATURES].copy()

    model_xgb = XGBClassifier(
        device="cuda",
        max_depth=3,  
        colsample_bytree=0.7129400756425178, 
        subsample=0.8185881823156917, 
        n_estimators=20_000, 
        learning_rate=0.04425768131771064,  
        eval_metric="auc", 
        early_stopping_rounds=50, 
        objective='binary:logistic',
        scale_pos_weight=1.5379160847615545,  
        min_child_weight=4,
        enable_categorical=True,
        gamma=3.1330719334577584
    )
    model_xgb.fit(
        x_train, y_train,
        eval_set=[(x_valid, y_valid)],  
        verbose=100
    )


    # INFER OOF (Probabilities -> Binary)
    oof_xgb[test_index] = (model_xgb.predict_proba(x_valid)[:, 1] > 0.5).astype(int)
    # INFER TEST (Probabilities -> Average Probs)
    pred_efs += model_xgb.predict_proba(x_test)[:, 1]

# COMPUTE AVERAGE TEST PREDS
pred_efs = (pred_efs / FOLDS > 0.5).astype(int)

# EVALUATE PERFORMANCE
accuracy = accuracy_score(train["efs"], oof_xgb)
f1 = f1_score(train["efs"], oof_xgb)
roc_auc = roc_auc_score(train["efs"], oof_xgb)
train_global = train.copy()
test_global = test.copy()
train_global["efs_xgb_pred"] = oof_xgb
test_global["efs_xgb_pred"] = pred_efs
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import TensorDataset
from warnings import filterwarnings

filterwarnings('ignore')


def get_X_cat(df, cat_cols, transformers=None):
    """
    Apply a specific categorical data transformer or a LabelEncoder if None.
    すべてのデータをスキャンし、ユニークなカテゴリを取得して、対応する整数を割り当てる。
    """
    if transformers is None:
        transformers = [LabelEncoder().fit(df[col]) for col in cat_cols]
    return transformers, np.array(
        [transformer.transform(df[col]) for col, transformer in zip(cat_cols, transformers)]
    ).T


def preprocess_data(train, val):
    """
    Standardize numerical variables and transform (Label-encode) categoricals.
    Fill NA values with mean for numerical.
    Create torch dataloaders to prepare data for training and evaluation.
    数値変数を標準化し、カテゴリ変数を変換（ラベルエンコード）する。
数値データの NA（欠損値）は平均値で埋める。
学習と評価のために、データを準備する torch のデータローダーを作成する。
    """
    X_cat_train, X_cat_val, numerical, transformers = get_categoricals(train, val)
    scaler = StandardScaler()
    # 数値データの NA（欠損値）は平均値で埋める。
    imp = SimpleImputer(missing_values=np.nan, strategy='mean', add_indicator=True)
    X_num_train = imp.fit_transform(train[numerical])
    X_num_train = scaler.fit_transform(X_num_train)
    X_num_val = imp.transform(val[numerical])
    X_num_val = scaler.transform(X_num_val)
    dl_train = init_dl(X_cat_train, X_num_train, train, training=True)
    dl_val = init_dl(X_cat_val, X_num_val, val)
    return X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers


def get_categoricals(train, val):
    """
    Remove constant categorical columns and transform them using LabelEncoder.
    Return the label-transformers for each categorical column, categorical dataframes and numerical columns.
    """
    categorical_cols, numerical = get_feature_types(train)
    remove = []
    for col in categorical_cols:
        if train[col].nunique() == 1:
            remove.append(col)
        ind = ~val[col].isin(train[col])
        if ind.any():
            val.loc[ind, col] = np.nan
    categorical_cols = [col for col in categorical_cols if col not in remove]
    transformers, X_cat_train = get_X_cat(train, categorical_cols)
    _, X_cat_val = get_X_cat(val, categorical_cols, transformers)
    return X_cat_train, X_cat_val, numerical, transformers


def init_dl(X_cat, X_num, df, training=False):
    """
    Initialize data loaders with 4 dimensions : categorical dataframe, numerical dataframe and target values (efs and efs_time).
    Notice that efs_time is log-transformed.
    Fix batch size to 2048 and return dataloader for training or validation depending on training value.
    """
    ds_train = TensorDataset(
        torch.tensor(X_cat, dtype=torch.long),
        torch.tensor(X_num, dtype=torch.float32),
        torch.tensor(df.efs_time.values, dtype=torch.float32).log(),
        torch.tensor(df.efs.values, dtype=torch.long)
    )
    bs = 2048
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=bs, pin_memory=True, shuffle=training)
    return dl_train


def get_feature_types(train):
    """
    Utility function to return categorical and numerical column names.
    """
    categorical_cols = [col for i, col in enumerate(train.columns) if ((train[col].dtype == "object") | (2 < train[col].nunique() < 25))]
    RMV = ["ID", "efs", "efs_time", "y"]
    FEATURES = [c for c in train.columns if not c in RMV]
    numerical = [i for i in FEATURES if i not in categorical_cols]
    return categorical_cols, numerical


def add_features(df):
    """
    Create some new features to help the model focus on specific patterns.
    """
    # sex_match = df.sex_match.astype(str)
    # sex_match = sex_match.str.split("-").str[0] == sex_match.str.split("-").str[1]
    # df['sex_match_bool'] = sex_match
    # df.loc[df.sex_match.isna(), 'sex_match_bool'] = np.nan
    # df['big_age'] = df.age_at_hct > 16
    # df.loc[df.year_hct == 2019, 'year_hct'] = 2020
    df['is_cyto_score_same'] = (df['cyto_score'] == df['cyto_score_detail']).astype(int)
    # df['strange_age'] = df.age_at_hct == 0.044
    # df['age_bin'] = pd.cut(df.age_at_hct, [0, 0.0441, 16, 30, 50, 100])
    # df['age_ts'] = df.age_at_hct / df.donor_age
    df['year_hct'] -= 2000
    
    return df


def load_data():
    """
    Load data and add features.
    """
    test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
    test = add_features(test)
    test["efs_xgb_pred"] = test_global["efs_xgb_pred"]
    
    print("Test shape:", test.shape)
    train = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
    train = add_features(train)
    train["efs_xgb_pred"] = train_global["efs_xgb_pred"]
    print("Train shape:", train.shape)
    return test, train


In [ ]:
import functools
from typing import List

import pytorch_lightning as pl
import numpy as np
import torch
from lifelines.utils import concordance_index
from pytorch_lightning.cli import ReduceLROnPlateau
from pytorch_tabular.models.common.layers import ODST
from torch import nn
from pytorch_lightning.utilities import grad_norm


class CatEmbeddings(nn.Module):
    """
    Embedding module for the categorical dataframe.
    """
    def __init__(
        self,
        projection_dim: int,
        categorical_cardinality: List[int],
        embedding_dim: int
    ):
        """
        projection_dim: The dimension of the final output after projecting the concatenated embeddings into a lower-dimensional space.
        categorical_cardinality: A list where each element represents the number of unique categories (cardinality) in each categorical feature.
        embedding_dim: The size of the embedding space for each categorical feature.
        self.embeddings: list of embedding layers for each categorical feature.
        self.projection: sequential neural network that goes from the embedding to the output projection dimension with GELU activation.
        """
        super(CatEmbeddings, self).__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(cardinality, embedding_dim)
            for cardinality in categorical_cardinality
        ])
        self.projection = nn.Sequential(
            nn.Linear(embedding_dim * len(categorical_cardinality), projection_dim),
            nn.GELU(),
            nn.Linear(projection_dim, projection_dim)
        )

    def forward(self, x_cat):
        """
        Apply the projection on concatened embeddings that contains all categorical features.
        """
        x_cat = [embedding(x_cat[:, i]) for i, embedding in enumerate(self.embeddings)]
        x_cat = torch.cat(x_cat, dim=1)
        return self.projection(x_cat)


class NN(nn.Module):
    """
    Train a model on both categorical embeddings and numerical data.
    """
    def __init__(
            self,
            continuous_dim: int,
            categorical_cardinality: List[int],
            embedding_dim: int,
            projection_dim: int,
            hidden_dim: int,
            dropout: float = 0
    ):
        """
        continuous_dim: The number of continuous features.
        categorical_cardinality: A list of integers representing the number of unique categories in each categorical feature.
        embedding_dim: The dimensionality of the embedding space for each categorical feature.
        projection_dim: The size of the projected output space for the categorical embeddings.
        hidden_dim: The number of neurons in the hidden layer of the MLP.
        dropout: The dropout rate applied in the network.
        self.embeddings: previous embeddings for categorical data.
        self.mlp: defines an MLP model with an ODST layer followed by batch normalization and dropout.
        self.out: linear output layer that maps the output of the MLP to a single value
        self.dropout: defines dropout
        Weights initialization with xavier normal algorithm and biases with zeros.
        """
        super(NN, self).__init__()
        self.embeddings = CatEmbeddings(projection_dim, categorical_cardinality, embedding_dim)
        self.mlp = nn.Sequential(
            ODST(projection_dim + continuous_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout)
        )
        self.out = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

        # initialize weights
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x_cat, x_cont):
        """
        Create embedding layers for categorical data, concatenate with continous variables.
        Add dropout and goes through MLP and return raw output and 1-dimensional output as well.
        """
        x = self.embeddings(x_cat)
        x = torch.cat([x, x_cont], dim=1)
        x = self.dropout(x)
        x = self.mlp(x)
        return self.out(x), x


@functools.lru_cache
def combinations(N):
    """
    calculates all possible 2-combinations (pairs) of a tensor of indices from 0 to N-1, 
    and caches the result using functools.lru_cache for optimization
    """
    ind = torch.arange(N)
    comb = torch.combinations(ind, r=2)
    if torch.cuda.is_available():
        return comb.cuda()
    else:
        return comb.to("cpu")  # ここを変更
        


class LitNN(pl.LightningModule):
    """
    Main Model creation and losses definition to fully train the model.
    """
    def __init__(
            self,
            continuous_dim: int,
            categorical_cardinality: List[int],
            embedding_dim: int,
            projection_dim: int,
            hidden_dim: int,
            lr: float = 1e-3,
            dropout: float = 0.2,
            weight_decay: float = 1e-3,
            aux_weight: float = 0.1,
            margin: float = 0.5,
            race_index: int = 0
    ):
        """
        continuous_dim: The number of continuous input features.
        categorical_cardinality: A list of integers, where each element corresponds to the number of unique categories for each categorical feature.
        embedding_dim: The dimension of the embeddings for the categorical features.
        projection_dim: The dimension of the projected space after embedding concatenation.
        hidden_dim: The size of the hidden layers in the feedforward network (MLP).
        lr: The learning rate for the optimizer.
        dropout: Dropout probability to avoid overfitting.
        weight_decay: The L2 regularization term for the optimizer.
        aux_weight: Weight used for auxiliary tasks.
        margin: Margin used in some loss functions.
        race_index: An index that refer to race_group in the input data.
        """
        super(LitNN, self).__init__()
        self.save_hyperparameters()

        # Creates an instance of the NN model defined above
        self.model = NN(
            continuous_dim=self.hparams.continuous_dim,
            categorical_cardinality=self.hparams.categorical_cardinality,
            embedding_dim=self.hparams.embedding_dim,
            projection_dim=self.hparams.projection_dim,
            hidden_dim=self.hparams.hidden_dim,
            dropout=self.hparams.dropout
        )
        self.targets = []

        # Defines a small feedforward neural network that performs an auxiliary task with 1-dimensional output
        self.aux_cls = nn.Sequential(
            nn.Linear(self.hparams.hidden_dim, self.hparams.hidden_dim // 3),
            nn.GELU(),
            nn.Linear(self.hparams.hidden_dim // 3, 1)
        )

    def on_before_optimizer_step(self, optimizer):
        """
        Compute the 2-norm for each layer
        If using mixed precision, the gradients are already unscaled here
        """
        norms = grad_norm(self.model, norm_type=2)
        self.log_dict(norms)

    def forward(self, x_cat, x_cont):
        """
        Forward pass that outputs the 1-dimensional prediction and the embeddings (raw output)
        """
        x, emb = self.model(x_cat, x_cont)
        return x.squeeze(1), emb

    def training_step(self, batch, batch_idx):
        """
        defines how the model processes each batch of data during training.
        A batch is a combination of : categorical data, continuous data, efs_time (y) and efs event.
        y_hat is the efs_time prediction on all data and aux_pred is auxiliary prediction on embeddings.
        Calculates loss and race_group loss on full data.
        Auxiliary loss is calculated with an event mask, ignoring efs=0 predictions and taking the average.
        Returns loss and aux_loss multiplied by weight defined above.
        """
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        aux_pred = self.aux_cls(emb).squeeze(1)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        aux_loss = nn.functional.mse_loss(aux_pred, y, reduction='none')
        aux_mask = efs == 1
        aux_loss = (aux_loss * aux_mask).sum() / aux_mask.sum()
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("race_loss", race_loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        self.log("aux_loss", aux_loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        return loss + aux_loss * self.hparams.aux_weight

    def get_full_loss(self, efs, x_cat, y, y_hat):
        """
        Output loss and race_group loss.
        """
        loss = self.calc_loss(y, y_hat, efs)
        race_loss = self.get_race_losses(efs, x_cat, y, y_hat)
        loss += 0.1 * race_loss
        return loss, race_loss

    def get_race_losses(self, efs, x_cat, y, y_hat):
        """
        Calculate loss for each race_group based on deviation/variance.
        """
        races = torch.unique(x_cat[:, self.hparams.race_index])
        race_losses = []
        for race in races:
            ind = x_cat[:, self.hparams.race_index] == race
            race_losses.append(self.calc_loss(y[ind], y_hat[ind], efs[ind]))
        race_loss = sum(race_losses) / len(race_losses)
        races_loss_std = sum((r - race_loss)**2 for r in race_losses) / len(race_losses)
        return torch.sqrt(races_loss_std)

    def calc_loss(self, y, y_hat, efs):
        """
        Most important part of the model : loss function used for training.
        We face survival data with event indicators along with time-to-event.

        This function computes the main loss by the following the steps :
        * create all data pairs with "combinations" function (= all "two subjects" combinations)
        * make sure that we have at least 1 event in each pair
        * convert y to +1 or -1 depending on the correct ranking
        * loss is computed using a margin-based hinge loss
        * mask is applied to ensure only valid pairs are being used (censored data can't be ranked with event in some cases)
        * average loss on all pairs is returned
        """
        N = y.shape[0]
        comb = combinations(N)
        comb = comb[(efs[comb[:, 0]] == 1) | (efs[comb[:, 1]] == 1)]
        pred_left = y_hat[comb[:, 0]]
        pred_right = y_hat[comb[:, 1]]
        y_left = y[comb[:, 0]]
        y_right = y[comb[:, 1]]
        y = 2 * (y_left > y_right).int() - 1
        loss = nn.functional.relu(-y * (pred_left - pred_right) + self.hparams.margin)
        mask = self.get_mask(comb, efs, y_left, y_right)
        loss = (loss.double() * (mask.double())).sum() / mask.sum()
        return loss

    def get_mask(self, comb, efs, y_left, y_right):
        """
        Defines all invalid comparisons :
        * Case 1: "Left outlived Right" but Right is censored
        * Case 2: "Right outlived Left" but Left is censored
        Masks for case 1 and case 2 are combined using |= operator and inverted using ~ to create a "valid pair mask"
        """
        left_outlived = y_left >= y_right
        left_1_right_0 = (efs[comb[:, 0]] == 1) & (efs[comb[:, 1]] == 0)
        mask2 = (left_outlived & left_1_right_0)
        right_outlived = y_right >= y_left
        right_1_left_0 = (efs[comb[:, 1]] == 1) & (efs[comb[:, 0]] == 0)
        mask2 |= (right_outlived & right_1_left_0)
        mask2 = ~mask2
        mask = mask2
        return mask

    def validation_step(self, batch, batch_idx):
        """
        This method defines how the model processes each batch during validation
        """
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        self.targets.append([y, y_hat.detach(), efs, x_cat[:, self.hparams.race_index]])
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def on_validation_epoch_end(self):
        """
        At the end of the validation epoch, it computes and logs the concordance index
        """
        cindex, metric = self._calc_cindex()
        self.log("cindex", metric, on_epoch=True, prog_bar=True, logger=True)
        self.log("cindex_simple", cindex, on_epoch=True, prog_bar=True, logger=True)
        self.targets.clear()

    def _calc_cindex(self):
        """
        Calculate c-index accounting for each race_group or global.
        """
        y = torch.cat([t[0] for t in self.targets]).cpu().numpy()
        y_hat = torch.cat([t[1] for t in self.targets]).cpu().numpy()
        efs = torch.cat([t[2] for t in self.targets]).cpu().numpy()
        races = torch.cat([t[3] for t in self.targets]).cpu().numpy()
        metric = self._metric(efs, races, y, y_hat)
        cindex = concordance_index(y, y_hat, efs)
        return cindex, metric

    def _metric(self, efs, races, y, y_hat):
        """
        Calculate c-index accounting for each race_group
        """
        metric_list = []
        for race in np.unique(races):
            y_ = y[races == race]
            y_hat_ = y_hat[races == race]
            efs_ = efs[races == race]
            metric_list.append(concordance_index(y_, y_hat_, efs_))
        metric = float(np.mean(metric_list) - np.sqrt(np.var(metric_list)))
        return metric

    def test_step(self, batch, batch_idx):
        """
        Same as training step but to log test data
        """
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        self.targets.append([y, y_hat.detach(), efs, x_cat[:, self.hparams.race_index]])
        self.log("test_loss", loss)
        return loss

    def on_test_epoch_end(self) -> None:
        """
        At the end of the test epoch, calculates and logs the concordance index for the test set
        """
        cindex, metric = self._calc_cindex()
        self.log("test_cindex", metric, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_cindex_simple", cindex, on_epoch=True, prog_bar=True, logger=True)
        self.targets.clear()


    def configure_optimizers(self):
        """
        configures the optimizer and learning rate scheduler:
        * Optimizer: Adam optimizer with weight decay (L2 regularization).
        * Scheduler: Cosine Annealing scheduler, which adjusts the learning rate according to a cosine curve.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        scheduler_config = {
            "scheduler": torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer,
                T_max=45,
                eta_min=6e-3
            ),
            "interval": "epoch",
            "frequency": 1,
            "strict": False,
        }

        return {"optimizer": optimizer, "lr_scheduler": scheduler_config}

In [ ]:
import json
import pytorch_lightning as pl
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import torch
from pytorch_lightning.callbacks import LearningRateMonitor, TQDMProgressBar
from pytorch_lightning.callbacks import StochasticWeightAveraging
from sklearn.model_selection import StratifiedKFold

pl.seed_everything(42)

# デバイスを決定（GPUがあればcuda、なければcpu）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def main(hparams):
    test, train_original = load_data()
    test['efs_time'] = 1
    test['efs'] = 1
    oof_nn_pairwise = np.zeros(len(train_original))
    test_pred = np.zeros(test.shape[0])
    categorical_cols, numerical = get_feature_types(train_original)
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    
    for i, (train_index, test_index) in enumerate(
        kf.split(train_original, train_original.race_group.astype(str) + (train_original.age_at_hct == 0.044).astype(str))
    ):
        tt = train_original.copy()
        train = tt.iloc[train_index]
        val = tt.iloc[test_index]
        X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers = preprocess_data(train, val)
        
        model = train_final(X_num_train, dl_train, dl_val, transformers, categorical_cols=categorical_cols).to(device)

        oof_pred, _ = model.eval()(
            torch.tensor(X_cat_val, dtype=torch.long).to(device),
            torch.tensor(X_num_val, dtype=torch.float32).to(device)
        )
        oof_nn_pairwise[test_index] = oof_pred.detach().cpu().numpy()

        # Create submission
        train = tt.iloc[train_index]
        X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers = preprocess_data(train, test)

        pred, _ = model.eval()(
            torch.tensor(X_cat_val, dtype=torch.long).to(device),
            torch.tensor(X_num_val, dtype=torch.float32).to(device)
        )
        test_pred += pred.detach().cpu().numpy()
    
    return -test_pred, -oof_nn_pairwise


def train_final(X_num_train, dl_train, dl_val, transformers, hparams=None, categorical_cols=None):
    if hparams is None:
        hparams = {
            "embedding_dim": 16,
            "projection_dim": 112,
            "hidden_dim": 56,
            "lr": 0.06464861983337984,
            "dropout": 0.05463240181423116,
            "aux_weight": 0.26545778308743806,
            "margin": 0.2588153271003354,
            "weight_decay": 0.0002773544957610778
        }
    
    model = LitNN(
        continuous_dim=X_num_train.shape[1],
        categorical_cardinality=[len(t.classes_) for t in transformers],
        race_index=categorical_cols.index("race_group"),
        **hparams
    ).to(device)  # モデルを適切なデバイスに移動

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        monitor="val_loss",  # val_loss が最小のモデルを保存
        save_top_k=1,  # 最も良いモデルだけを保存
    )

    trainer = pl.Trainer(
        accelerator=device.type,  # 自動で GPU or CPU を選択
        max_epochs=60,
        log_every_n_steps=6,
        callbacks=[
            checkpoint_callback,
            LearningRateMonitor(logging_interval='epoch'),
            TQDMProgressBar(),
            StochasticWeightAveraging(swa_lrs=1e-5, swa_epoch_start=45, annealing_epochs=15)
        ],
    )

    trainer.fit(model, dl_train)
    trainer.test(model, dl_val)

    return model.eval().to(device)  # モデルを適切なデバイスに移動


In [ ]:
hparams = None
pairwise_ranking_pred, pairwise_ranking_oof = main(hparams)

y_true = train[["ID","efs","efs_time","race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = pairwise_ranking_oof
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"\nPairwise ranking NN CV =",m)

# Update predictions with classifier mask
pairwise_ranking_oof[oof_xgb == 1] += 0.2
y_pred["prediction"] = pairwise_ranking_oof
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"\nPairwise ranking NN with classifier mask -> CV =",m)

pairwise_ranking_pred[pred_efs == 1] += 0.2

In [ ]:
# subm_data = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv")
# subm_data['prediction'] = pairwise_ranking_pred
# subm_data.to_csv('submission2.csv', index=False)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import polars as pl #ライブラリのインポートに非常に時間がかかりそうだったので割愛
# import plotly.colors as pc
# import plotly.express as px
# import plotly.graph_objects as go
# import plotly.io as pio
# pio.renderers.default = 'iframe'

# 生存関数推定モデル
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
from lifelines import NelsonAalenFitter
from lifelines.utils import concordance_index #C-indexの計算

# sklearn
import sklearn.base
from sklearn.base import _fit_context
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

# 予測モデル
import lightgbm as lgb
import optuna
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from scipy.stats import rankdata

#NN用
import torch
from lifelines import KaplanMeierFitter, NelsonAalenFitter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import TensorDataset

import functools
from typing import List

import pytorch_lightning as pl
import numpy as np
import torch
from lifelines.utils import concordance_index
from pytorch_lightning.cli import ReduceLROnPlateau
from pytorch_tabular.models.common.layers import ODST
from torch import nn
from pytorch_lightning.utilities import grad_norm

import json
import joblib
import os
from datetime import datetime
import xgboost as xgb


import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, TQDMProgressBar
from pytorch_lightning.callbacks import StochasticWeightAveraging
from sklearn.model_selection import StratifiedKFold

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">データインポート</p>

In [ ]:
# INPUT_DIRにディレクトリを指定
INPUT_DIR = "xxx"
pd.options.display.max_columns = None

In [ ]:
if "COLAB_GPU" in os.environ:
  import sys
  sys.path.append('/content/drive/MyDrive/Kaggle/20250215_CIBMTR')


  print("Google Colab で実行中")
  from google.colab import drive
  drive.mount('/content/drive')
  # CSVファイルのパスを指定
  csv_file_path = "/content/drive/MyDrive/Kaggle/20250215_CIBMTR/death_rate.csv"
  # 現在の日時を取得してフォーマット
  current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  # ディレクトリ名を作成
  directory_name = f"/content/drive/MyDrive/Kaggle/20250215_CIBMTR/model_{current_datetime}"

  # ディレクトリのパスを指定
  directory_path = Path(directory_name)

  # ディレクトリを作成
  directory_path.mkdir(parents=True, exist_ok=True)

  train_path = Path('/content/drive/MyDrive/Kaggle/20250215_CIBMTR/data/train.csv')
  test_path = Path('/content/drive/MyDrive/Kaggle/20250215_CIBMTR/data/test.csv')
  subm_path = Path('/content/drive/MyDrive/Kaggle/20250215_CIBMTR/data/sample_submission.csv')


elif "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
  !pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
  !pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  !pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
  !pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
  !pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl




  # CSVファイルのパスを指定
  csv_file_path = "/kaggle/input/world-age-death-rate/death_rate.csv"
  # 現在の日時を取得してフォーマット
  current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  # ディレクトリ名を作成
  directory_name = f"model_{current_datetime}"

  # ディレクトリのパスを指定
  directory_path = Path(directory_name)

  # ディレクトリを作成
  directory_path.mkdir(parents=True, exist_ok=True)
  print("Kaggle Notebooks で実行中")

  train_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
  test_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')
  subm_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv')
else:
  print("ローカル環境または他の環境で実行中")

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">パラメータ設定【CFG】</p>

重みや正則化パラメータは自分で決定

In [ ]:
# クラス変数（クラス全体で共有される変数）の定義
class CFG:
    train_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/train.csv') 
    test_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/test.csv') 
    subm_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv') 
    # train_path = INPUT_DIR + "/train.csv"
    # test_path = INPUT_DIR + "/test.csv"
    # subm_path = INPUT_DIR + "/sample_submission.csv"

    # colorscale = 'red' # 可視化に使用するカラースケールの指定
    color = '#A2574F' # 可視化に使用するカラーパレットの指定

    early_stop = 300 # 早期終了のパラメータ（一定エポック改善がない場合に終了）
    penalizer = 0.01 # 正則化パラメータの設定
    n_splits = 5 # クロスバリデーションの分割数

    weights = [
        2,
        1,
        6,
        3,
        6,
        3,
        6,
        6,
        6,
    ] # 特定のモデルやデータに対する重みのリスト


    # GPUが利用可能か判定
    use_gpu = torch.cuda.is_available()
    
    # CatBoost (二値分類)
    ctb_params_efs = {
        "loss_function": "Logloss",
        'learning_rate': 0.03,
        'random_state': 42,
        'task_type': 'GPU' if use_gpu else 'CPU',  # GPUが使えればGPU、使えなければCPU
        'num_trees': 6000,
        'reg_lambda': 8.0,
        'depth': 8,
        'eval_metric': "Logloss",
    }
    
    # CatBoost (回帰)
    ctb_params = {
        'loss_function': 'RMSE',
        'learning_rate': 0.03,
        'random_state': 42,
        'task_type': 'GPU' if use_gpu else 'CPU',
        'num_trees': 6000,
        'reg_lambda': 8.0,
        'depth': 8,
    }
    
    # LightGBM (二値分類)
    lgb_params_efs = {
        'objective': 'binary',
        'min_child_samples': 32,
        'num_iterations': 6000,
        'learning_rate': 0.03,
        'extra_trees': True,
        'reg_lambda': 8.0,
        'reg_alpha': 0.1,
        'num_leaves': 64,
        'metric': 'logloss',
        'max_depth': 8,
        'device': 'gpu' if use_gpu else 'cpu',
        'max_bin': 128,
        'verbose': -1,
        'seed': 42,
        **({'gpu_use_dp': True} if use_gpu else {})  # GPU使用時は `gpu_use_dp=True` を追加
    }
    
    # LightGBM (回帰)
    lgb_params = {
        'objective': 'regression',
        'min_child_samples': 32,
        'num_iterations': 6000,
        'learning_rate': 0.03,
        'extra_trees': True,
        'reg_lambda': 8.0,
        'reg_alpha': 0.1,
        'num_leaves': 64,
        'metric': 'binary_logloss',
        'max_depth': 8,
        'device': 'gpu' if use_gpu else 'cpu',
        'max_bin': 128,
        'verbose': -1,
        'seed': 42,
        **({'gpu_use_dp': True} if use_gpu else {})
    }
    
    # XGBoost (回帰)
    xgb_params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'learning_rate': 0.03,
        'max_depth': 7,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'lambda': 7.0,
        'alpha': 0.0,
        'n_estimators': 6000,
        'tree_method': 'gpu_hist' if use_gpu else 'hist',
        'random_state': 42,
    }
    
    # XGBoost (二値分類)
    xgb_params_efs = {
        'objective': "binary:logistic",
        'eval_metric': 'logloss',
        'learning_rate': 0.03,
        'max_depth': 7,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'lambda': 7.0,
        'alpha': 0.0,
        'n_estimators': 6000,
        'tree_method': 'gpu_hist' if use_gpu else 'hist',
        'random_state': 42,
    }
    
    # Coxモデル (Depthwise)
    cox1_params = {
        'grow_policy': 'Depthwise',
        'min_child_samples': 8,
        'loss_function': 'Cox',
        'learning_rate': 0.03,
        'random_state': 42,
        'task_type': 'GPU' if use_gpu else 'CPU',
        'num_trees': 6000,
        'reg_lambda': 8.0,
        'depth': 8,
    }
    
    # Coxモデル (Lossguide)
    cox2_params = {
        'grow_policy': 'Lossguide',
        'min_child_samples': 2,
        'loss_function': 'Cox',
        'learning_rate': 0.03,
        'random_state': 42,
        'task_type': 'GPU' if use_gpu else 'CPU',
        'num_trees': 6000,
        'reg_lambda': 8.0,
        'num_leaves': 32,
        'depth': 8,
    }


<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">特徴量エンジニアリング【FE】</p>

In [ ]:
class FE:
    def __init__(self):
        pass # 初期化メソッドには何も処理を行わない

    def _load_data(self, path):
        df = pd.read_csv(path)
        return df

    def _update_hla_columns(self, df): # "hla_high_xxx"、"hla_low_xxx"系の特徴量について、既知のデータを用いずに自分で作成
        df['hla_nmdp_6'] = (
            df['hla_match_a_low'].fillna(0)
            + df['hla_match_b_low'].fillna(0)
            + df['hla_match_drb1_high'].fillna(0)
        )
        df['hla_low_res_6'] = (
            df['hla_match_a_low'].fillna(0)
            + df['hla_match_b_low'].fillna(0)
            + df['hla_match_drb1_low'].fillna(0)
        )
        df['hla_high_res_6'] = (
            df['hla_match_a_high'].fillna(0)
            + df['hla_match_b_high'].fillna(0)
            + df['hla_match_drb1_high'].fillna(0)
        )
        df['hla_low_res_8'] = (
            df['hla_match_a_low'].fillna(0)
            + df['hla_match_b_low'].fillna(0)
            + df['hla_match_c_low'].fillna(0)
            + df['hla_match_drb1_low'].fillna(0)
        )
        df['hla_high_res_8'] = (
            df['hla_match_a_high'].fillna(0)
            + df['hla_match_b_high'].fillna(0)
            + df['hla_match_c_high'].fillna(0)
            + df['hla_match_drb1_high'].fillna(0)
        )
        df['hla_low_res_10'] = (
            df['hla_match_a_low'].fillna(0)
            + df['hla_match_b_low'].fillna(0)
            + df['hla_match_c_low'].fillna(0)
            + df['hla_match_drb1_low'].fillna(0)
            + df['hla_match_dqb1_low'].fillna(0)
        )
        df['hla_high_res_10'] = (
            df['hla_match_a_high'].fillna(0)
            + df['hla_match_b_high'].fillna(0)
            + df['hla_match_c_high'].fillna(0)
            + df['hla_match_drb1_high'].fillna(0)
            + df['hla_match_dqb1_high'].fillna(0)
        )
        return df

    def _cast_datatypes(self, df): # 欠損値の補完
        num_cols = [
            'hla_high_res_8', 'hla_low_res_8', 'hla_high_res_6', 'hla_low_res_6',
            'hla_high_res_10', 'hla_low_res_10', 'hla_match_dqb1_high',
            'hla_match_dqb1_low', 'hla_match_drb1_high', 'hla_match_drb1_low',
            'hla_nmdp_6', 'year_hct', 'hla_match_a_high', 'hla_match_a_low',
            'hla_match_b_high', 'hla_match_b_low', 'hla_match_c_high',
            'hla_match_c_low', 'donor_age', 'age_at_hct', 'comorbidity_score',
            'karnofsky_score', 'efs', 'efs_time'
        ]
        for col in df.columns:
            if col in num_cols:
                df[col] = df[col].fillna(-1).astype('float32') # 数値型の場合は欠損値を-1で補完
            else:
                df[col] = df[col].fillna('Unknown').astype('category') # カテゴリ型の場合は欠損値を'Unknown'で補完
        df['ID'] = df['ID'].astype('int32') # データ型の変更
        return df
    
    def add_features(df):
        sex_match = df.sex_match.astype(str)
        sex_match = sex_match.str.split("-").str[0] == sex_match.str.split("-").str[1]
        df['sex_match_bool'] = sex_match
        df.loc[df.sex_match.isna(), 'sex_match_bool'] = np.nan
        df.loc[df.year_hct == 2019, 'year_hct'] = 2020
        df['is_cyto_score_same'] = (df['cyto_score'] == df['cyto_score_detail']).astype(int)
        df['age_bin'] = pd.cut(df.age_at_hct, [0, 0.0441, 16, 30, 50, 100])
        df['age_ts'] = df.age_at_hct / df.donor_age
        df['age_comorbidity'] = df['age_at_hct'] * df['comorbidity_score']
        df['age_karnofsky'] = df['age_at_hct'] * df['karnofsky_score']
        df['karnofsky_squared'] = df['karnofsky_score'] ** 2
        df['cos_year'] = np.cos(df['year_hct'] * (2 * np.pi) / 100)
        df['diff_age_vs_donor'] = df['age_at_hct'] - df['donor_age']
        # sex_one: 性別一致情報から患者の性別を抽出
        df['sex_one'] = df['sex_match'].str[0]  # 最初の文字を抽出
        # sex_two: 性別一致情報からドナーの性別を抽出
        df['sex_two'] = df['sex_match'].str[2]  # 3番目の文字を抽出
        # SameSex: 患者とドナーの性別が一致しているか
        df['same_sex'] = (df['sex_one'] == df['sex_two']).astype(int)

        
        return df

    def info(self, df): #データフレームのメモリ使用量を確認
        print(f'\nShape of dataframe: {df.shape}')
        mem = df.memory_usage().sum() / 1024**2
        print('Memory usage: {:.2f} MB\n'.format(mem))
        print(df.head())

    def apply_fe(self, path): # 上記関数の適用
        df = self._load_data(path) # pathのデータを読み込む
        df = self._update_hla_columns(df) # hla_high_xxx、hla_low_xxx系の特徴量を追加
        df = self._cast_datatypes(df) # 欠損値の補完
        self.info(df) # データフレームのメモリ確認
        cat_cols = [col for col in df.columns if df[col].dtype == 'category']
        return df, cat_cols

In [ ]:
# インスタンス作成
fe = FE()

In [ ]:
# 
df_train, cat_cols = fe.apply_fe(CFG.train_path)

In [ ]:
test_data, _ = fe.apply_fe(CFG.test_path)

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">可視化【EDA】</p>

In [ ]:
class EDA:
    def __init__(self, color, data):
        # self._colorscale = sns.color_palette(colorscale)
        self._color = color
        self.data = data

    def _template(self, ax, title):
        ax.set_title(title, fontsize=16, color=self._color, ha='center')
        # ax.set_facecolor('rgba(247, 230, 202, 1)')
        ax.grid(True, axis='x', color='grey', linestyle='-', linewidth=0.5)
        ax.grid(True, axis='y', color='grey', linestyle='-', linewidth=0.5)
        ax.tick_params(axis='both', colors=self._color)
        ax.set_xlabel('Values', color=self._color)
        ax.set_ylabel('Count', color=self._color)
        return ax

    def distribution_plot(self, col, title):
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.histplot(self.data[col], bins=100, color=self._color, ax=ax)
        ax = self._template(ax, f'{title}')
        plt.show()

    def bar_chart(self, col):
        value_counts = self.data[col].value_counts().reset_index()
        value_counts.columns = [col, 'count']

        fig, ax = plt.subplots(figsize=(8, 6))
        sns.barplot(data=value_counts, x='count', y=col, ax=ax)

        ax = self._template(ax, f'{col}')
        ax.set_xlabel('Count')
        ax.set_ylabel('')
        plt.show()

    def _plot_cv(self, scores, title, metric='Stratified C-Index'):
        fold_scores = [round(score, 3) for score in scores]
        mean_score = round(np.mean(scores), 3)

        fig, ax = plt.subplots(figsize=(8, 6))

        ax.scatter(range(1, len(fold_scores) + 1), fold_scores, color=self._color, s=100, label='Fold Scores', marker='D')

        ax.plot([1, len(fold_scores)], [mean_score, mean_score], color='#B22222', linestyle='--', label=f'Mean: {mean_score:.3f}')
        
        ax.set_title(f'{title} | Cross-validation Mean {metric} Score: {mean_score}', fontsize=16, color=self._color, ha='center')
        ax.set_xlabel('Fold', color=self._color)
        ax.set_ylabel(f'{metric} Score', color=self._color)
        
        ax.legend()
        ax.set_xticks(range(1, len(fold_scores) + 1))
        ax.set_ylim(min(fold_scores) - 0.05, max(fold_scores) + 0.05)

        ax = self._template(ax, f'{title} Cross-validation')
        plt.show()

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">生存確率（目的変数）の推定【Targets】</p>

特徴的な点  
- カテゴリ変数にone hot encodingを実施
- cvにおいて一つの値しか取らないカラムを削除

In [ ]:
import torch
class Targets:
    def __init__(self, data, cat_cols, penalizer, n_splits):
        self.data = data
        self.cat_cols = cat_cols
        self._length = len(self.data)
        self._penalizer = penalizer
        self._n_splits = n_splits

    def _prepare_cv(self): # cvのためのデータ分割
        oof_preds = np.zeros(self._length)
        cv = KFold(n_splits=self._n_splits, shuffle=True, random_state=42)
        return cv, oof_preds

    def score(self, df_true, df_prediction, event_label, interval_label, prediction_label): #層別C-indexの計算
        # ID列を削除
        del df_true["ID"]
        del df_prediction["ID"]

        merged_df = pd.concat([df_true, df_prediction], axis=1)
        merged_df.reset_index(inplace=True)
        merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups) # 人種ごとに該当するデータのインデックスを辞書形式で保管
        metric_list = []
        for race in merged_df_race_dict.keys(): #人種ごと
            indices = sorted(merged_df_race_dict[race]) #該当する人種のインデックスを取得
            merged_df_race = merged_df.iloc[indices] # 該当するインデックスのデータを取得
            c_index_race = concordance_index( # concordance_indexはefs, efs_time, predicton（生存確率）からC-indexを計算
                merged_df_race[interval_label],
                -merged_df_race[prediction_label], # 符号を反転（通常 C-Index は「リスクが高いほど短命」と仮定するため）
                merged_df_race[event_label])
            metric_list.append(c_index_race)
        return float(np.mean(metric_list)-np.sqrt(np.var(metric_list)))

    def validate_model(self, preds, title):
        y_true = self.data[['ID', 'efs', 'efs_time', 'race_group']].copy()
        y_pred = self.data[['ID']].copy()
        y_pred['prediction'] = preds
        c_index_score = self.score(y_true.copy(), y_pred.copy(), "efs", "efs_time", "prediction")
        print(f'Overall Stratified C-Index Score for {title}: {c_index_score:.4f}')

        return c_index_score

    def create_target1(self): # Cox比例ハザードモデル
        '''
        Constant columns are dropped if they exist in a fold. Otherwise, the code produces error:
        delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: 
        https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
        '''
        cv, oof_preds = self._prepare_cv() #データ分割
        data = pd.get_dummies(self.data, columns=self.cat_cols, drop_first=True).drop('ID', axis=1) # カテゴリ変数にone hot encodingを実施

        for fold, (train_index, valid_index) in enumerate(cv.split(data), 1):
            df_train = data.iloc[train_index]
            valid_data = data.iloc[valid_index]
            unique_columns = df_train.nunique() > 1
            removed_columns = df_train.columns[~unique_columns] # 一つの値しか取らないカラムを削除
            df_train = df_train.loc[:, unique_columns]
            valid_data = valid_data[df_train.columns]
            print(f"Fold {fold}:")
            print(f"Removed columns: {list(removed_columns)}") # 削除されたカラムとその数を表示
            print(f"Number of removed columns: {len(removed_columns)}")
            cph = CoxPHFitter(penalizer=self._penalizer) # Cox比例ハザードモデルのインスタンス作成（penalizer：正則化の強さ（過学習を防止する目的））
            cph.fit(df_train, duration_col='efs_time', event_col='efs')
            oof_preds[valid_index] = cph.predict_partial_hazard(valid_data)

        self.data['target1'] = oof_preds
        c_index_score = self.validate_model(oof_preds, 'Cox')
        return self.data

    def create_target2(self): # Kaplan-Meier生存関数モデル
        cv, oof_preds = self._prepare_cv() #データ分割

        for train_index, valid_index in cv.split(self.data):
            df_train = self.data.iloc[train_index]
            valid_data = self.data.iloc[valid_index]
            kmf = KaplanMeierFitter()
            kmf.fit(durations=df_train['efs_time'], event_observed=df_train['efs'])
            oof_preds[valid_index] = kmf.survival_function_at_times(valid_data['efs_time']).values

        self.data['target2'] = oof_preds
        c_index_score = self.validate_model(oof_preds, 'Kaplan-Meier')

        return self.data

    def create_target3(self): # Nelson-Aalen生存関数モデル
        cv, oof_preds = self._prepare_cv()
        for train_index, valid_index in cv.split(self.data):
            df_train = self.data.iloc[train_index]
            valid_data = self.data.iloc[valid_index]
            naf = NelsonAalenFitter()
            naf.fit(durations=df_train['efs_time'], event_observed=df_train['efs'])
            oof_preds[valid_index] = -naf.cumulative_hazard_at_times(valid_data['efs_time']).values

        self.data['target3'] = oof_preds
        self.validate_model(oof_preds, 'Nelson-Aalen')
        return self.data

    def create_target4(self): # 
        self.data['target4'] = self.data.efs_time.copy()
        self.data.loc[self.data.efs == 0, 'target4'] *= -1
        return self.data

    def create_target5(self):
        # GPU が利用可能なら "cuda" を、なければ "cpu" を使用
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # pandas のカラム "efs" と "efs_time" を PyTorch テンソルに変換
        efs = torch.tensor(self.data["efs"].values, device=device)
        efs_time = torch.tensor(self.data["efs_time"].values, device=device)
        
        N = len(self.data)
        
        # 全ての (i, j) (i < j) の組み合わせを生成（torch.combinations は全ペアを返す）
        pairs = torch.combinations(torch.arange(N, device=device), r=2)
        i_idx, j_idx = pairs[:, 0], pairs[:, 1]
        
        # 各ペアの値を抽出
        efs_i, efs_j = efs[i_idx], efs[j_idx]
        time_i, time_j = efs_time[i_idx], efs_time[j_idx]
        
        # 各サンプルの勝利数を格納するテンソル（初期値0）
        win_counts = torch.zeros(N, device=device)
        
        # (1,1) のペアの場合：時間が長い方が勝者
        condition_11 = (efs_i == 1) & (efs_j == 1)
        win_counts.index_add_(0, i_idx[condition_11 & (time_i > time_j)], torch.ones((((condition_11) & (time_i > time_j)).sum(),), device=device))
        win_counts.index_add_(0, j_idx[condition_11 & (time_j > time_i)], torch.ones((((condition_11) & (time_j > time_i)).sum(),), device=device))
        
        # (1,0) の場合：efs_i==1 かつ efs_j==0 で、time_j > time_i のとき、j が勝者
        condition_10 = (efs_i == 1) & (efs_j == 0) & (time_j > time_i)
        win_counts.index_add_(0, j_idx[condition_10], torch.ones((condition_10.sum(),), device=device))
        
        # (0,1) の場合：efs_i==0 かつ efs_j==1 で、time_i > time_j のとき、i が勝者
        condition_01 = (efs_i == 0) & (efs_j == 1) & (time_i > time_j)
        win_counts.index_add_(0, i_idx[condition_01], torch.ones((condition_01.sum(),), device=device))
        
        # 結果を負の値に変換してデータフレームに格納
        self.data["target5"] = -win_counts.cpu().numpy()
        
        # 検証処理（元の処理と統一）
        self.validate_model(self.data["target5"], 'pair_wise')
    
        return self.data

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">モデル構築【MD】</p>

In [ ]:
class MD:
    def __init__(self,color, data, cat_cols, early_stop, penalizer, n_splits):
        self.eda = EDA(color, data) # EDAクラスのインスタンス作成
        self.targets = Targets(data, cat_cols, penalizer, n_splits) # Targetsクラスのインスタンス作成
        self.data = data
        self.cat_cols = cat_cols
        self.early_stop = early_stop
        self.efs_lgb_models = []
        self.efs_cbt_models = []
        self.efs_xgb_models = []

    def create_targets(self, ctb_params_efs, lgb_params_efs, xgb_params_efs, directory_path): # 4種類の生存確率
        self.data = self.targets.create_target1()
        self.data = self.targets.create_target2()
        self.data = self.targets.create_target3()
        self.data = self.targets.create_target4()
        self.data = self.targets.create_target5()

        self.data["efs_pred_lgb"] = 0  # 事前にカラムを用意（任意）cat
        self.data["efs_pred_cbt"] = 0  # 事前にカラムを用意（任意）
        self.data["efs_pred_xgb"] = 0  # 事前にカラムを用意（任意）cat
        # epsの予測モデルを追加
        for col in self.cat_cols:
            self.data[col] = self.data[col].astype('category')

        

        cv, oof_preds = self.targets._prepare_cv()

        print(self.data.columns)
        X = self.data.drop(['ID', 'efs', 'efs_time', 'target1', 'target2', 'target3', 'target4','target5',
                            "efs_pred_lgb", "efs_pred_cbt", "efs_pred_xgb"
                           ], axis=1, errors='ignore')
        y = self.data['efs']
        w =  1 + self.data["efs"]  # 最小1、最大2の重み

        models, fold_scores = [], []

        for fold, (train_index, valid_index) in enumerate(cv.split(X, y)):
            # 訓練データとバリデーションデータの分割
            X_train = X.iloc[train_index]
            X_valid = X.iloc[valid_index]
            y_train = y.iloc[train_index]
            y_valid = y.iloc[valid_index]
            w_train = w.iloc[train_index]

            # 未使用カテゴリを削除
            for col in X_train.select_dtypes(include=['category']).columns:
                X_train[col] = X_train[col].cat.remove_unused_categories()
                X_valid[col] = X_valid[col].cat.remove_unused_categories()

# lgb-------------------------------------------------------------------------------------------------

            model = lgb.LGBMClassifier(**lgb_params_efs)

            model.fit(
                X_train,
                y_train,
                sample_weight=w_train,
                eval_set=[(X_valid, y_valid)],
                eval_metric='logloss',
                callbacks=[lgb.early_stopping(self.early_stop, verbose=1), lgb.log_evaluation(period=1000)],
            )
            # モデルを保存
            joblib.dump(model, directory_path / f'lightgbm_model_efs_fold{fold}.pkl')

            self.efs_lgb_models.append(model)

            oof_preds[valid_index] = model.predict_proba(X_valid)[:, 1]

            self.data.loc[valid_index, "efs_pred_lgb"] = oof_preds[valid_index]

        print("lgb_efs完了")
# catboost-----------------------------------------------------------------------------------------------------
        for fold, (train_index, valid_index) in enumerate(cv.split(X, y)):
            # 訓練データとバリデーションデータの分割
            X_train = X.iloc[train_index]
            X_valid = X.iloc[valid_index]
            y_train = y.iloc[train_index]
            y_valid = y.iloc[valid_index]
            w_train = w.iloc[train_index]
            model = CatBoostClassifier(**ctb_params_efs, verbose=0, cat_features=self.cat_cols)

            model.fit(
                X_train,
                y_train,
                sample_weight=w_train,
                eval_set=(X_valid, y_valid),
                early_stopping_rounds=self.early_stop,
                verbose=1000,
            )
            model_file_path = directory_path / f'catboost_model_efs_fold{fold}.cbm'
            # モデルを保存
            model.save_model(model_file_path)
            self.efs_cbt_models.append(model)

            oof_preds[valid_index] = model.predict_proba(X_valid)[:, 1]
            self.data.loc[valid_index, "efs_pred_cbt"] = oof_preds[valid_index]
        print("ctb_efs完了")
# xgb-----------------------------------------------------------------------------------------------
        for fold, (train_index, valid_index) in enumerate(cv.split(X, y)):
            # 訓練データとバリデーションデータの分割
            X_train = X.iloc[train_index]
            X_valid = X.iloc[valid_index]
            y_train = y.iloc[train_index]
            y_valid = y.iloc[valid_index]
            w_train = w.iloc[train_index]
            X_train_xgb = xgb.DMatrix(X_train.copy(), label=y_train, weight=w_train, enable_categorical=True)
            X_valid_xgb = xgb.DMatrix(X_valid.copy(), label=y_valid, enable_categorical=True)

             # モデルの学習
            model = xgb.train(
                params=xgb_params_efs,
                dtrain=X_train_xgb,
                num_boost_round=10000,                      # 最大イテレーション数
                evals=[(X_train_xgb, 'train'), (X_valid_xgb, 'valid')],  # 学習データと検証データのセット
                early_stopping_rounds=self.early_stop,    # CatBoostのearly_stopping_roundsに相当
                verbose_eval=1000,                      # 学習ログの間隔
                )

             # モデルの保存
            model_file_path = directory_path / f'xgboost_model_efs_fold{fold}.model'

            # JSON形式で保存
            model.save_model(model_file_path.with_suffix(".json"))

            oof_preds[valid_index] = model.predict(X_valid_xgb)
            self.data.loc[valid_index, "efs_pred_xgb"] = oof_preds[valid_index]

            self.efs_xgb_models.append(model)

            # モデルの読み込み
            # model = xgb.Booster()
            # model.load_model("xgboost_model_target1_fold0.json")
        print("xgb_efs完了")

        
        return self.data

    def score(self, df_true, df_prediction, event_label, interval_label, prediction_label): #層別C-indexの計算
        # ID列を削除
        del df_true["ID"]
        del df_prediction["ID"]

        merged_df = pd.concat([df_true, df_prediction], axis=1)
        merged_df.reset_index(inplace=True)
        merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups) # 人種ごとに該当するデータのインデックスを辞書形式で保管
        metric_list = []
        for race in merged_df_race_dict.keys(): #人種ごと
            indices = sorted(merged_df_race_dict[race]) #該当する人種のインデックスを取得
            merged_df_race = merged_df.iloc[indices] # 該当するインデックスのデータを取得
            c_index_race = concordance_index( # concordance_indexはefs, efs_time, predicton（生存確率）からC-indexを計算
                merged_df_race[interval_label],
                -merged_df_race[prediction_label], # 符号を反転（通常 C-Index は「リスクが高いほど短命」と仮定するため）
                merged_df_race[event_label])
            metric_list.append(c_index_race)
        return float(np.mean(metric_list)-np.sqrt(np.var(metric_list)))

    def train_model(self, params, target, title, directory_path):
        for col in self.cat_cols:
            self.data[col] = self.data[col].astype('category')
        X = self.data.drop(['ID',
                            'efs',
                            'efs_time',
                            'target1',
                            'target2',
                            'target3',
                            'target4',
                            'target5',
                           ], axis=1)
        y = self.data[target]
        w =  1 + self.data["efs"]  # 最小1、最大2の重み
        models, fold_scores = [], []
        cv, oof_preds = self.targets._prepare_cv()
        for fold, (train_index, valid_index) in enumerate(cv.split(X, y)):
            X_train = X.iloc[train_index]
            X_valid = X.iloc[valid_index]
            y_train = y.iloc[train_index]
            y_valid = y.iloc[valid_index]
            w_train = w.iloc[train_index]
            if title.startswith('LightGBM'):
                model = lgb.LGBMRegressor(**params)
                model.fit(
                    X_train,
                    y_train,
                    sample_weight=w_train,
                    eval_set=[(X_valid, y_valid)],
                    eval_metric='rmse',
                    callbacks=[lgb.early_stopping(self.early_stop, verbose=0), lgb.log_evaluation(0)]
                )
                # モデルを保存
                joblib.dump(model, directory_path / f'lightgbm_model_{target}_fold{fold}.pkl')
            elif title.startswith('CatBoost'):
                model = CatBoostRegressor(**params, verbose=0, cat_features=self.cat_cols)
                model.fit(
                    X_train,
                    y_train,
                    sample_weight=w_train,
                    eval_set=(X_valid, y_valid),
                    early_stopping_rounds=self.early_stop, 
                    verbose=0
                )
                model_file_path = directory_path / f'catboost_model_{target}_fold{fold}.cbm'
                if target == 'target4':
                    if params['grow_policy'] == 'Depthwise':
                        model_file_path = directory_path / f'catboost_model_{target}_Cox1_fold{fold}.cbm'
                    else:
                        model_file_path = directory_path / f'catboost_model_{target}_Cox2_fold{fold}.cbm'
                # モデルを保存
                model.save_model(model_file_path)

            models.append(model) # モデルのリストに追加
            oof_preds[valid_index] = model.predict(X_valid) # 予測値を格納

            y_true_fold = self.data.iloc[valid_index][['ID', 'efs', 'efs_time', 'race_group']].copy()
            y_pred_fold = self.data.iloc[valid_index][['ID']].copy()
            y_pred_fold['prediction'] = oof_preds[valid_index] # 予測値を格納

            fold_score = self.score(y_true_fold, y_pred_fold, "efs", "efs_time", "prediction")
            fold_scores.append(fold_score) # スコアのリストに追加
        self.eda._plot_cv(fold_scores, title)
        c_index_score = self.targets.validate_model(oof_preds, title)
        return models, oof_preds

    def infer_model(self, data, models):
        data = data.drop(['ID'], axis=1)
        for col in self.cat_cols: # カテゴリ変数をcategory型に変換
            data[col] = data[col].astype('category')

        efs_pred_lgb = np.mean([model.predict(data) for model in self.efs_lgb_models], axis=0)
        efs_pred_cbt = np.mean([model.predict(data) for model in self.efs_cbt_models], axis=0)
        efs_pred_xgb = np.mean([model.predict(xgb.DMatrix(data, enable_categorical=True)) for model in self.efs_xgb_models], axis=0)

        data["efs_pred_lgb"] = efs_pred_lgb
        data["efs_pred_cbt"] = efs_pred_cbt
        data["efs_pred_xgb"] = efs_pred_xgb
        
        return np.mean([model.predict(data) for model in models], axis=0)

In [ ]:
# MDクラスのインスタンス作成
md = MD(CFG.color, df_train, cat_cols, CFG.early_stop, CFG.penalizer, CFG.n_splits)

In [ ]:
df_train = md.create_targets(CFG.ctb_params_efs, CFG.lgb_params_efs, CFG.xgb_params_efs, directory_path)

各推定量のC統計量のうち、Cox比例ハザードモデルものが低い原因（あくまでも推定）
- 過学習を防ぐ目的で`_penalizer`を追加しているため？？
> Overall Stratified C-Index Score for Cox: 0.6564  
> Overall Stratified C-Index Score for Kaplan-Meier: 0.9983  
> Overall Stratified C-Index Score for Nelson-Aalen: 0.9983  


In [ ]:
md.eda.bar_chart('race_group')

In [ ]:
md.eda.distribution_plot('target1', 'Cox Target')

In [ ]:
md.eda.distribution_plot('target2', 'Kaplan-Meier Target')

In [ ]:
md.eda.distribution_plot('target3', 'Nelson-Aalen Target')

In [ ]:
md.eda.distribution_plot('target4', 'Cox-Loss Target')

In [ ]:
md.eda.distribution_plot('target5', 'Pairwise Target')

In [ ]:
fe.info(df_train)

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">Cox比例ハザードモデルによる予測</p>

- 目的変数
    - target1
- 予測モデル
    - lgbm
    - catboost

In [ ]:
# CatBoost
ctb1_models, ctb1_oof_preds = md.train_model(CFG.ctb_params, target='target1', title='CatBoost', directory_path=directory_path)

In [ ]:
# LightGBM
lgb1_models, lgb1_oof_preds = md.train_model(CFG.lgb_params, target='target1', title='LightGBM', directory_path=directory_path)

In [ ]:
ctb1_preds = md.infer_model(test_data, ctb1_models)

In [ ]:
lgb1_preds = md.infer_model(test_data, lgb1_models)

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">カプラン・マイヤー推定量による予測</p>

- 目的変数
    - target2
- 予測モデル
    - lgbm
    - catboost

In [ ]:
ctb2_models, ctb2_oof_preds = md.train_model(CFG.ctb_params, target='target2', title='CatBoost', directory_path=directory_path)

In [ ]:
lgb2_models, lgb2_oof_preds = md.train_model(CFG.lgb_params, target='target2', title='LightGBM', directory_path=directory_path)

In [ ]:
ctb2_preds = md.infer_model(test_data, ctb2_models)

In [ ]:
lgb2_preds = md.infer_model(test_data, lgb2_models)

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">ネルソン・アーレン推定量による予測</p>

- 目的変数
    - target3
- 予測モデル
    - lgbm
    - catboost

In [ ]:
ctb3_models, ctb3_oof_preds = md.train_model(CFG.ctb_params, target='target3', title='CatBoost', directory_path=directory_path)

In [ ]:
lgb3_models, lgb3_oof_preds = md.train_model(CFG.lgb_params, target='target3', title='LightGBM', directory_path=directory_path)

In [ ]:
ctb3_preds = md.infer_model(test_data, ctb3_models)

In [ ]:
lgb3_preds = md.infer_model(test_data, lgb3_models)

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">Cox-Lossによる予測</p>

- 目的変数
    - target4
- 予測モデル
    - catboost
        - 木の深さ優先
        - 損失最小優先

In [ ]:
# 木の深さ優先
cox1_models, cox1_oof_preds = md.train_model(CFG.cox1_params, target='target4', title='CatBoost', directory_path=directory_path)

In [ ]:
# 損失最小優先
cox2_models, cox2_oof_preds = md.train_model(CFG.cox2_params, target='target4', title='CatBoost', directory_path=directory_path)

In [ ]:
cox1_preds = md.infer_model(test_data, cox1_models)

In [ ]:
cox2_preds = md.infer_model(test_data, cox2_models)

In [ ]:
ctb5_models, ctb5_oof_preds = md.train_model(CFG.ctb_params, target='target5', title='CatBoost', directory_path=directory_path)

In [ ]:
lgb5_models, lgb5_oof_preds = md.train_model(CFG.lgb_params, target='target5', title='LightGBM', directory_path=directory_path)

In [ ]:
ctb5_preds = md.infer_model(test_data, ctb5_models)

In [ ]:
lgb5_preds = md.infer_model(test_data, lgb5_models)

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">ニューラルネットワーク</p>

In [ ]:
def transform_target(df_train):
    #efs_timeを対数変換（結局これが目的変数）
    df_train['y'] = np.log(1 + df_train.efs_time)
    return df_train

#ラベルエンコーディングを行い，カテゴリカル変数を整数変数に変換
def get_X_cat(df_train, cat_cols, transformers=None):
    if transformers is None:
        transformers = [LabelEncoder().fit(df_train[col]) for col in cat_cols]
    return transformers, np.array(
        [transformer.transform(df_train[col]) for col, transformer in zip(cat_cols, transformers)]
    ).T


def preprocess_data(df_train, val):
    X_cat_train, X_cat_val, numerical, transformers = get_categoricals(df_train, val)
    #標準化
    scaler = StandardScaler()
    #欠損値を平均値で補完
    imp = SimpleImputer(missing_values=np.nan, strategy='mean', add_indicator=True)
    X_num_train = imp.fit_transform(df_train[numerical])
    X_num_train = scaler.fit_transform(X_num_train)
    X_num_val = imp.transform(val[numerical])
    X_num_val = scaler.transform(X_num_val)
    dl_train = init_dl(X_cat_train, X_num_train, df_train, training=True)
    dl_val = init_dl(X_cat_val, X_num_val, val)
    return X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers


def get_categoricals(df_train, val):
    categorical_cols, numerical = get_feature_types(df_train)
    remove = []
    for col in categorical_cols:
        if df_train[col].nunique() == 1:
            remove.append(col)
        ind = ~val[col].isin(df_train[col])
        if ind.any():
            val.loc[ind, col] = np.nan
    categorical_cols = [col for col in categorical_cols if col not in remove]
    transformers, X_cat_train = get_X_cat(df_train, categorical_cols)
    _, X_cat_val = get_X_cat(val, categorical_cols, transformers)
    return X_cat_train, X_cat_val, numerical, transformers


def init_dl(X_cat, X_num, df_train, training=False):
    ds_train = TensorDataset(
        torch.tensor(X_cat, dtype=torch.long),
        torch.tensor(X_num, dtype=torch.float32),
        torch.tensor(df_train.efs_time.values, dtype=torch.float32).log(),
        torch.tensor(df_train.efs.values, dtype=torch.long)
    )
    bs = 2048
    # if not training:
    #     bs = 2048 * 8
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=bs, pin_memory=True, shuffle=training)
    return dl_train


def get_feature_types(df_train):
    categorical_cols = [col for i, col in enumerate(df_train.columns) if ((df_train[col].dtype == "object") | (2 < df_train[col].nunique() < 25))]
    RMV = ["ID", "efs", "efs_time", "y"]
    FEATURES = [c for c in df_train.columns if not c in RMV]
    print(f"There are {len(FEATURES)} FEATURES: {FEATURES}")
    numerical = [i for i in FEATURES if i not in categorical_cols]
    return categorical_cols, numerical

#dataフレームに特徴量を追加
def add_features(df):
    sex_match = df.sex_match.astype(str)
    sex_match = sex_match.str.split("-").str[0] == sex_match.str.split("-").str[1]
    df['sex_match_bool'] = sex_match
    df.loc[df.sex_match.isna(), 'sex_match_bool'] = np.nan
    df.loc[df.year_hct == 2019, 'year_hct'] = 2020
    df['is_cyto_score_same'] = (df['cyto_score'] == df['cyto_score_detail']).astype(int)
    df['age_bin'] = pd.cut(df.age_at_hct, [0, 0.0441, 16, 30, 50, 100])
    df['age_ts'] = df.age_at_hct / df.donor_age
    df['age_comorbidity'] = df['age_at_hct'] * df['comorbidity_score']
    df['age_karnofsky'] = df['age_at_hct'] * df['karnofsky_score']
    df['karnofsky_squared'] = df['karnofsky_score'] ** 2
    df['cos_year'] = np.cos(df['year_hct'] * (2 * np.pi) / 100)
    df['diff_age_vs_donor'] = df['age_at_hct'] - df['donor_age']
    # sex_one: 性別一致情報から患者の性別を抽出
    df['sex_one'] = df['sex_match'].str[0]  # 最初の文字を抽出
    # sex_two: 性別一致情報からドナーの性別を抽出
    df['sex_two'] = df['sex_match'].str[2]  # 3番目の文字を抽出
    # SameSex: 患者とドナーの性別が一致しているか
    df['same_sex'] = (df['sex_one'] == df['sex_two']).astype(int)
    return df

In [ ]:
class CatEmbeddings(nn.Module):
    def __init__(
        self,
        projection_dim: int,
        categorical_cardinality: List[int],
        embedding_dim: int
    ):
        super(CatEmbeddings, self).__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(cardinality, embedding_dim)
            for cardinality in categorical_cardinality
        ])
        self.projection = nn.Sequential(
            nn.Linear(embedding_dim * len(categorical_cardinality), projection_dim),
            nn.GELU(),
            nn.Linear(projection_dim, projection_dim)
        )

    def forward(self, x_cat):
        x_cat = [embedding(x_cat[:, i]) for i, embedding in enumerate(self.embeddings)]
        x_cat = torch.cat(x_cat, dim=1)
        return self.projection(x_cat)


class NN(nn.Module):
    def __init__(
            self,
            continuous_dim: int,
            categorical_cardinality: List[int],
            embedding_dim: int,
            projection_dim: int,
            hidden_dim: int,
            dropout: float = 0
    ):
        super(NN, self).__init__()
        self.embeddings = CatEmbeddings(projection_dim, categorical_cardinality, embedding_dim)
        self.mlp = nn.Sequential(
            ODST(projection_dim + continuous_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout)
        )
        self.out = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

        # initialize weights
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x_cat, x_cont):
        x = self.embeddings(x_cat)
        x = torch.cat([x, x_cont], dim=1)
        x = self.dropout(x)
        x = self.mlp(x)
        return self.out(x), x


@functools.lru_cache
def combinations(N):
    ind = torch.arange(N)
    comb = torch.combinations(ind, r=2)
    return comb  # CPU 上でそのまま使う


class LitNN(pl.LightningModule):
    def __init__(
            self,
            continuous_dim: int,
            categorical_cardinality: List[int],
            embedding_dim: int,
            projection_dim: int,
            hidden_dim: int,
            lr: float = 1e-3,
            dropout: float = 0.2,
            weight_decay: float = 1e-3,
            aux_weight: float = 0.1,
            margin: float = 0.5,
            race_index: int = 0
    ):
        super(LitNN, self).__init__()
        self.save_hyperparameters()
        self.model = NN(
            continuous_dim=self.hparams.continuous_dim,
            categorical_cardinality=self.hparams.categorical_cardinality,
            embedding_dim=self.hparams.embedding_dim,
            projection_dim=self.hparams.projection_dim,
            hidden_dim=self.hparams.hidden_dim,
            dropout=self.hparams.dropout
        )
        self.targets = []
        self.aux_cls = nn.Sequential(
            nn.Linear(self.hparams.hidden_dim, self.hparams.hidden_dim // 3),
            nn.GELU(),
            nn.Linear(self.hparams.hidden_dim // 3, 1)
        )

    def on_before_optimizer_step(self, optimizer):
        # Compute the 2-norm for each layer
        # If using mixed precision, the gradients are already unscaled here
        norms = grad_norm(self.model, norm_type=2)
        self.log_dict(norms)

    def forward(self, x_cat, x_cont):
        x, emb = self.model(x_cat, x_cont)
        return x.squeeze(1), emb

    def training_step(self, batch, batch_idx):
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        aux_pred = self.aux_cls(emb).squeeze(1)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        aux_loss = nn.functional.mse_loss(aux_pred, y, reduction='none')
        aux_mask = efs == 1
        aux_loss = (aux_loss * aux_mask).sum() / aux_mask.sum()
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("race_loss", race_loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        self.log("aux_loss", aux_loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        return loss + aux_loss * self.hparams.aux_weight

    def get_full_loss(self, efs, x_cat, y, y_hat):
        loss = self.calc_loss(y, y_hat, efs)
        race_loss = self.get_race_losses(efs, x_cat, y, y_hat)
        loss += 0.1 * race_loss
        return loss, race_loss

    def get_race_losses(self, efs, x_cat, y, y_hat):
        races = torch.unique(x_cat[:, self.hparams.race_index])
        race_losses = []
        for race in races:
            ind = x_cat[:, self.hparams.race_index] == race
            race_losses.append(self.calc_loss(y[ind], y_hat[ind], efs[ind]))
        race_loss = sum(race_losses) / len(race_losses)
        races_loss_std = sum((r - race_loss)**2 for r in race_losses) / len(race_losses)
        return torch.sqrt(races_loss_std)

    def calc_loss(self, y, y_hat, efs):
        N = y.shape[0]
        comb = combinations(N)
        comb = comb[(efs[comb[:, 0]] == 1) | (efs[comb[:, 1]] == 1)]
        pred_left = y_hat[comb[:, 0]]
        pred_right = y_hat[comb[:, 1]]
        y_left = y[comb[:, 0]]
        y_right = y[comb[:, 1]]
        y = 2 * (y_left > y_right).int() - 1
        loss = nn.functional.relu(-y * (pred_left - pred_right) + self.hparams.margin)
        mask = self.get_mask(comb, efs, y_left, y_right)
        loss = (loss.double() * (mask.double())).sum() / mask.sum()
        return loss

    def get_mask(self, comb, efs, y_left, y_right):
        # mask1 = (efs[comb[:, 0]] == 1) | (efs[comb[:, 1]] == 1)
        left_outlived = y_left >= y_right
        left_1_right_0 = (efs[comb[:, 0]] == 1) & (efs[comb[:, 1]] == 0)
        mask2 = (left_outlived & left_1_right_0)
        right_outlived = y_right >= y_left
        right_1_left_0 = (efs[comb[:, 1]] == 1) & (efs[comb[:, 0]] == 0)
        mask2 |= (right_outlived & right_1_left_0)
        mask2 = ~mask2
        mask = mask2
        return mask

    def validation_step(self, batch, batch_idx):
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        self.targets.append([y, y_hat.detach(), efs, x_cat[:, self.hparams.race_index]])
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def on_validation_epoch_end(self):
        cindex, metric = self._calc_cindex()
        self.log("cindex", metric, on_epoch=True, prog_bar=True, logger=True)
        self.log("cindex_simple", cindex, on_epoch=True, prog_bar=True, logger=True)
        self.targets.clear()

    def _calc_cindex(self):
        y = torch.cat([t[0] for t in self.targets]).cpu().numpy()
        y_hat = torch.cat([t[1] for t in self.targets]).cpu().numpy()
        efs = torch.cat([t[2] for t in self.targets]).cpu().numpy()
        races = torch.cat([t[3] for t in self.targets]).cpu().numpy()
        metric = self._metric(efs, races, y, y_hat)
        cindex = concordance_index(y, y_hat, efs)
        return cindex, metric

    def _metric(self, efs, races, y, y_hat):
        metric_list = []
        for race in np.unique(races):
            y_ = y[races == race]
            y_hat_ = y_hat[races == race]
            efs_ = efs[races == race]
            metric_list.append(concordance_index(y_, y_hat_, efs_))
        metric = float(np.mean(metric_list) - np.sqrt(np.var(metric_list)))
        return metric

    def test_step(self, batch, batch_idx):
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        self.targets.append([y, y_hat.detach(), efs, x_cat[:, self.hparams.race_index]])
        self.log("test_loss", loss)
        return loss

    def on_test_epoch_end(self) -> None:
        cindex, metric = self._calc_cindex()
        self.log("test_cindex", metric, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_cindex_simple", cindex, on_epoch=True, prog_bar=True, logger=True)
        self.targets.clear()


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        scheduler_config = {
            "scheduler": torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer,
                T_max=45,
                eta_min=6e-3
            ),
            "interval": "epoch",
            "frequency": 1,
            "strict": False,
        }

        return {"optimizer": optimizer, "lr_scheduler": scheduler_config}

In [ ]:
pl.seed_everything(42)

def train_final(X_num_train, dl_train, dl_val, transformers, hparams=None, categorical_cols=None):

    # ハイパーパラメータのデフォルト値設定
    if hparams is None:
        hparams = {
            "embedding_dim": 16,
            "projection_dim": 112,
            "hidden_dim": 56,
            "lr": 0.06464861983337984,
            "dropout": 0.05463240181423116,
            "aux_weight": 0.26545778308743806,
            "margin": 0.2588153271003354,
            "weight_decay": 0.0002773544957610778
        }

    # モデルインスタンスの生成
    model = LitNN(
        continuous_dim=X_num_train.shape[1],
        categorical_cardinality=[len(t.classes_) for t in transformers],
        race_index=categorical_cols.index("race_group"),
        **hparams
    )

    # チェックポイントコールバック：検証ロス（val_loss）が最小のモデルを保存
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        monitor="val_loss",
        mode="min",
        save_top_k=1,
        filename="best-checkpoint"
    )

    # Trainerの初期化
    trainer = pl.Trainer(
        accelerator='cpu',  # GPUが使用可能なら "gpu" に変更
        max_epochs=60,
        callbacks=[
            checkpoint_callback,
            LearningRateMonitor(logging_interval='epoch'),
            TQDMProgressBar(),
            StochasticWeightAveraging(swa_lrs=1e-5, swa_epoch_start=45, annealing_epochs=15)
        ],
    )

    # 学習の実施（引数名を明示的にして分かりやすく）
    trainer.fit(model, train_dataloaders=dl_train, val_dataloaders=dl_val)

    # チェックポイントから最良モデルを復元（best_model_pathが存在する場合）
    best_model_path = checkpoint_callback.best_model_path
    if best_model_path:
        print(f"Loading best model from: {best_model_path}")
        model = LitNN.load_from_checkpoint(best_model_path)
    else:
        print("No checkpoint found. Using last model state.")

    # 検証データでテスト実行（結果表示などが必要ならここで利用）
    trainer.test(model, dataloaders=dl_val)

    # 評価モードへ切替え
    model.eval()
    return model


def main(hparams):
    FE_test = FE()
    test= FE_test._load_data(CFG.test_path)
    test=add_features(test)
    FE_train = FE()
    train_original = FE_train._load_data(CFG.train_path)
    train_original=add_features(train_original)
    test['efs_time'] = 1
    test['efs'] = 1

    # 学習データの行数に合わせたOOF配列を用意
    oof_nn_preds = np.zeros(train_original.shape[0])
    test_nn_preds = np.zeros(test.shape[0])
    
    # StratifiedKFoldのセットアップ（ここでは例として5fold）
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for fold, (train_index, valid_index) in enumerate(
        kf.split(train_original, train_original.race_group.astype(str) + (train_original.age_at_hct == 0.044).astype(str))
    ):
        # foldごとにデータを分割
        train = train_original.iloc[train_index].copy()
        valid = train_original.iloc[valid_index].copy()
        
        # データ前処理（数値部分・カテゴリ部分の整形等）
        X_cat_valid, X_num_train, X_num_valid, dl_train, dl_valid, transformers = preprocess_data(train, valid)
        categorical_cols, _ = get_feature_types(train)
        
        # モデルの学習
        model = train_final(X_num_train, dl_train, dl_valid, transformers, hparams, categorical_cols=categorical_cols)
        
        # 検証データに対する予測
        # ※入力テンソルの作成（GPU使用の場合はto(device)等を追加）
        with torch.no_grad():
            pred_valid, _ = model.cpu().eval()(
                torch.tensor(X_cat_valid, dtype=torch.long),
                torch.tensor(X_num_valid, dtype=torch.float32)
            )
        # もともとのコードでは提出用に符号反転していたので同様に
        oof_nn_preds[valid_index] = -pred_valid.detach().cpu().numpy().reshape(-1)
        
        # testデータに対する予測（foldごとに同じ手順で処理）
        # ここでは train のうち fold 用の部分を使ってテストに対する予測を実施
        X_cat_test, _, X_num_test, _, _, _ = preprocess_data(train, test)
        with torch.no_grad():
            pred_test, _ = model.cpu().eval()(
                torch.tensor(X_cat_test, dtype=torch.long),
                torch.tensor(X_num_test, dtype=torch.float32)
            )
        test_nn_preds += -pred_test.detach().cpu().numpy().reshape(-1)
        
    # 各foldのtest予測の平均を最終のNNテスト予測とする
    test_nn_preds /= kf.n_splits

    
    # NNモデルの結果として、OOFとTest予測の両方を返す
    return oof_nn_preds, test_nn_preds


In [ ]:
nn_oof_preds, nn_test_preds = main(hparams=None)

# CIBMTR Yunbase


In [ ]:
!pip install -q --requirement /kaggle/input/yunbase/Yunbase/requirements.txt  \
--no-index --find-links file:/kaggle/input/yunbase/

!pip install -q /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install -q /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install -q /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install -q /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install -q /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

In [ ]:
source_file_path = '/kaggle/input/yunbase/Yunbase/baseline.py'
target_file_path = '/kaggle/working/baseline.py'
with open(source_file_path, 'r', encoding='utf-8') as file:
    content = file.read()
with open(target_file_path, 'w', encoding='utf-8') as file:
    file.write(content)

In [ ]:
from baseline import Yunbase
import pandas as pd#read csv,parquet
import numpy as np#for scientific computation of matrices
from  lightgbm import LGBMRegressor,LGBMClassifier,log_evaluation,early_stopping
from catboost import CatBoostRegressor,CatBoostClassifier
from xgboost import XGBRegressor,XGBClassifier
from lifelines import KaplanMeierFitter
import warnings#avoid some negligible errors
#The filterwarnings () method is used to set warning filters, which can control the output method and level of warning information.
warnings.filterwarnings('ignore')
import random#provide some function to generate random_seed.
#set random seed,to make sure model can be recurrented.
def seed_everything(seed):
    np.random.seed(seed)#numpy's random seed
    random.seed(seed)#python built-in random seed
seed_everything(seed=2025)

train=pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
test=pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
train_solution=train[['ID','efs','efs_time','race_group']].copy()

def logit(p):
    return np.log(p) - np.log(1 - p)
max_efs_time,min_efs_time=80,-100
train['efs_time']=train['efs_time']/(max_efs_time-min_efs_time)
train['efs_time']=train['efs_time'].apply(lambda x:logit(x))
train['efs_time']+=10
print(train['efs_time'].max(),train['efs_time'].min())

race2weight={'American Indian or Alaska Native':0.68,
'Asian':0.7,'Black or African-American':0.67,
'More than one race':0.68,
'Native Hawaiian or other Pacific Islander':0.66,
'White':0.64}
train['weight']=0.5*train['efs']+0.5
train['raceweight']=train['race_group'].apply(lambda x:race2weight.get(x,1))
train['weight']=train['weight']/train['raceweight']
train.drop(['raceweight'],axis=1,inplace=True)

train.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def transform_survival_probability(df, time_col='efs_time', event_col='efs'):

    kmf = KaplanMeierFitter()
    
    kmf.fit(df[time_col], event_observed=df[event_col])
    
    survival_probabilities = kmf.survival_function_at_times(df[time_col]).values.flatten()

    return survival_probabilities

race_group=sorted(train['race_group'].unique())
for race in race_group:
    train.loc[train['race_group']==race,"target"] = transform_survival_probability(train[train['race_group']==race], time_col='efs_time', event_col='efs')
    gap=0.7*(train.loc[(train['race_group']==race)&(train['efs']==0)]['target'].max()-train.loc[(train['race_group']==race)&(train['efs']==1)]['target'].min())/2
    train.loc[(train['race_group']==race)&(train['efs']==0),'target']-=gap

sns.histplot(data=train, x='target', hue='efs', element='step', stat='density', common_norm=False)
plt.legend(title='efs')
plt.title('Distribution of Target by EFS')
plt.xlabel('Target')
plt.ylabel('Density')
plt.show()

train.drop(['efs','efs_time'],axis=1,inplace=True)

In [ ]:
#nunique=2
nunique2=[col for col in train.columns if train[col].nunique()==2 and col!='efs']
#nunique<50
nunique50=[col for col in train.columns if train[col].nunique()<50 and col not in ['efs','weight']]+['age_group','dri_score_NA']

def FE(df):
    print("< deal with outlier >")
    df['nan_value_each_row'] = df.isnull().sum(axis=1)
    #year_hct=2020 only 4 rows.
    df['year_hct']=df['year_hct'].replace(2020,2019)
    df['age_group']=df['age_at_hct']//10
    #karnofsky_score 40 only 10 rows.
    df['karnofsky_score']=df['karnofsky_score'].replace(40,50)
    #hla_high_res_8=2 only 2 rows.
    df['hla_high_res_8']=df['hla_high_res_8'].replace(2,3)
    #hla_high_res_6=0 only 1 row.
    df['hla_high_res_6']=df['hla_high_res_6'].replace(0,2)
    #hla_high_res_10=3 only 1 row.
    df['hla_high_res_10']=df['hla_high_res_10'].replace(3,4)
    #hla_low_res_8=2 only 1 row.
    df['hla_low_res_8']=df['hla_low_res_8'].replace(2,3)
    df['dri_score']=df['dri_score'].replace('Missing disease status','N/A - disease not classifiable')
    df['dri_score_NA']=df['dri_score'].apply(lambda x:int('N/A' in str(x)))
    for col in ['diabetes','pulm_moderate','cardiac']:
        df.loc[df[col].isna(),col]='Not done'

    print("< cross feature >")
    df['donor_age-age_at_hct']=df['donor_age']-df['age_at_hct']
    df['comorbidity_score+karnofsky_score']=df['comorbidity_score']+df['karnofsky_score']
    df['comorbidity_score-karnofsky_score']=df['comorbidity_score']-df['karnofsky_score']
    df['comorbidity_score*karnofsky_score']=df['comorbidity_score']*df['karnofsky_score']
    df['comorbidity_score/karnofsky_score']=df['comorbidity_score']/df['karnofsky_score']
    
    print("< fillna >")
    df[nunique50]=df[nunique50].astype(str).fillna('NaN')
    
    print("< combine category feature >")
    for i in range(len(nunique2)):
        for j in range(i+1,len(nunique2)):
            df[nunique2[i]+nunique2[j]]=df[nunique2[i]].astype(str)+df[nunique2[j]].astype(str)
    
    print("< drop useless columns >")
    df.drop(['ID'],axis=1,inplace=True,errors='ignore')
    return df

combine_category_cols=[]
for i in range(len(nunique2)):
    for j in range(i+1,len(nunique2)):
        combine_category_cols.append(nunique2[i]+nunique2[j])  

total_category_feature=nunique50+combine_category_cols

target_stat=[]
for j in range(len(total_category_feature)):
   for col in ['donor_age','age_at_hct','target']:
    target_stat.append( (total_category_feature[j],col,['count','mean','max','std','skew']) )

num_folds=10

import torch

# GPUが利用可能か判定
use_gpu = torch.cuda.is_available()

# LightGBM のパラメータ設定
lgb_params = {
    "boosting_type": "gbdt",
    "metric": 'mae',
    'random_state': 2025,
    "max_depth": 9,
    "learning_rate": 0.1,
    "n_estimators": 768,
    "colsample_bytree": 0.6,
    "colsample_bynode": 0.6,
    "verbose": -1,
    "reg_alpha": 0.2,
    "reg_lambda": 5,
    "extra_trees": True,
    'num_leaves': 64,
    "max_bin": 255,
    'importance_type': 'gain',  # better than 'split'
    'device': 'gpu' if use_gpu else 'cpu',  # GPUが使えればGPU、使えなければCPU
}

# GPU使用時の追加設定 (LightGBM)
if use_gpu:
    lgb_params['gpu_use_dp'] = True

# CatBoost のパラメータ設定
cat_params = {
    'random_state': 2025,
    'eval_metric': 'MAE',
    'bagging_temperature': 0.50,
    'iterations': 650,
    'learning_rate': 0.1,
    'max_depth': 8,
    'l2_leaf_reg': 1.25,
    'min_data_in_leaf': 24,
    'random_strength': 0.25,
    'verbose': 0,
    'task_type': 'GPU' if use_gpu else 'CPU',  # GPUが使えればGPU、使えなければCPU
}

# XGBoost のパラメータ設定 (コメント解除で使用可能)
xgb_params = {
    'random_state': 2025,
    'n_estimators': 256,
    'learning_rate': 0.1,
    'max_depth': 6,
    'reg_alpha': 0.08,
    'reg_lambda': 0.8,
    'subsample': 0.95,
    'colsample_bytree': 0.6,
    'min_child_weight': 3,
    'early_stopping_rounds': 1024,
    'enable_categorical': True,
    'tree_method': 'gpu_hist' if use_gpu else 'hist',  # GPUが使えればGPU、使えなければCPU
}

yunbase=Yunbase(num_folds=num_folds,
                  models=[(LGBMRegressor(**lgb_params),'lgb'),
                          (CatBoostRegressor(**cat_params),'cat'),
                          # (XGBRegressor(**xgb_params),'xgb')
                         ],
                  FE=FE,
                  seed=2025,
                  objective='regression',
                  metric='mae',
                  target_col='target',
                  device='cpu',
                  # device='gpu',
                  one_hot_max=-1,
                  early_stop=1000,
                  cross_cols=['donor_age','age_at_hct'],
                  target_stat=target_stat,
                  use_data_augmentation=True,
                  use_scaler=True,
                  log=250,
                  plot_feature_importance=True,
                  #print metric score when model training
                  use_eval_metric=False,
)
yunbase.fit(train,category_cols=nunique2)

In [ ]:
import pandas.api.types
from lifelines.utils import concordance_index

class ParticipantVisibleError(Exception):
    pass

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    
    del solution[row_id_column_name]
    del submission[row_id_column_name]
    
    event_label = 'efs'
    interval_label = 'efs_time'
    prediction_label = 'prediction'
    for col in submission.columns:
        if not pandas.api.types.is_numeric_dtype(submission[col]):
            raise ParticipantVisibleError(f'Submission column {col} must be a number')
    # Merging solution and submission dfs on ID
    merged_df = pd.concat([solution, submission], axis=1)
    merged_df.reset_index(inplace=True)
    merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)
    metric_list = []
    for race in merged_df_race_dict.keys():
        # Retrieving values from y_test based on index
        indices = sorted(merged_df_race_dict[race])
        merged_df_race = merged_df.iloc[indices]
        # Calculate the concordance index
        c_index_race = concordance_index(
                        merged_df_race[interval_label],
                        -merged_df_race[prediction_label],
                        merged_df_race[event_label])
        metric_list.append(c_index_race)
    return float(np.mean(metric_list)-np.sqrt(np.var(metric_list)))

weights = [0.5,0.5]

lgb_prediction=np.load(f"Yunbase_info/lgb_seed{yunbase.seed}_repeat0_fold{yunbase.num_folds}_{yunbase.target_col}.npy")
lgb_prediction=pd.DataFrame({'ID':train_solution['ID'],'prediction':lgb_prediction})
print(f"lgb_score:{score(train_solution.copy(),lgb_prediction.copy(),row_id_column_name='ID')}")
# xgb_prediction=np.load(f"Yunbase_info/xgb_seed{yunbase.seed}_repeat0_fold{yunbase.num_folds}_{yunbase.target_col}.npy")
# xgb_prediction=pd.DataFrame({'ID':train_solution['ID'],'prediction':xgb_prediction})
# print(f"xgb_score:{score(train_solution.copy(),xgb_prediction.copy(),row_id_column_name='ID')}")
cat_prediction=np.load(f"Yunbase_info/cat_seed{yunbase.seed}_repeat0_fold{yunbase.num_folds}_{yunbase.target_col}.npy")
cat_prediction=pd.DataFrame({'ID':train_solution['ID'],'prediction':cat_prediction})
print(f"cat_score:{score(train_solution.copy(),cat_prediction.copy(),row_id_column_name='ID')}")

y_preds=[
    lgb_prediction.copy(),
    # xgb_prediction.copy(),
    cat_prediction.copy()
]

final_prediction=lgb_prediction.copy()
final_prediction['prediction']=0
for i in range(len(y_preds)):
    final_prediction['prediction']+=weights[i]*y_preds[i]['prediction']
metric=score(train_solution.copy(),final_prediction.copy(),row_id_column_name='ID')
print(f"final_CV:{metric}")

In [ ]:
test_preds=yunbase.predict(test,weights=weights)
yunbase.target_col='prediction'
yunbase.submit("/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv",test_preds,
               save_name='submission1'
              )

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">アンサンブル</p>

In [ ]:
oof_preds = [
    ctb1_oof_preds, 
    lgb1_oof_preds, 
    ctb2_oof_preds, 
    lgb2_oof_preds, 
    ctb3_oof_preds, 
    lgb3_oof_preds,
    cox1_oof_preds,
    cox2_oof_preds,
    ctb5_oof_preds, 
    lgb5_oof_preds,
    nn_oof_preds,
    np.array(pairwise_ranking_oof),
    final_prediction['prediction'].to_numpy()
]

In [ ]:
# sub2 = pd.read_csv('/kaggle/working/submission2.csv')


preds = [
    ctb1_preds, 
    lgb1_preds, 
    ctb2_preds, 
    lgb2_preds, 
    ctb3_preds, 
    lgb3_preds,
    cox1_preds,
    cox2_preds,
    ctb5_preds, 
    lgb5_preds,
    nn_test_preds,
    np.array(pairwise_ranking_pred),
    test_preds,
]

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">アンサンブル結果の精度評価</p>

In [ ]:
# oof_preds（各予測モデル×）の中身について順位付け（値の小さい順）した結果を格納
ranked_oof_preds = np.array([rankdata(p) for p in oof_preds])

In [ ]:
import optuna
# 目的関数の定義
def objective(trial):
    # 各モデルの重みを提案 (0.0～1.0 の範囲)
    weights = [trial.suggest_float(f'weight_{i}', 0.05, 1.0) for i in range(len(ranked_oof_preds))]

    # 重みの正規化（合計が1になるようにスケーリング）
    weights = np.array(weights)
    weights /= np.sum(weights)

    # アンサンブル予測を計算 (加重平均)
    ensemble_oof_preds = np.dot(weights, ranked_oof_preds)

    curent_score = md.targets.validate_model(ensemble_oof_preds, 'Ensemble Model')

    return curent_score  # 最大化が目標

# Optunaによる最適化
study = optuna.create_study(direction='maximize')  # AUCを最大化
study.optimize(objective, n_trials=200)

# 最適な重み
print("Best Weights:", study.best_params)
print("Best AUC:", study.best_value)

CFG.weights = [1 for _ in range(len(ranked_oof_preds))]
np.array(CFG.weights)

for i in range(len(CFG.weights)):
    CFG.weights[i] = study.best_params[f'weight_{i}']

In [ ]:
ensemble_oof_preds = np.dot(CFG.weights, ranked_oof_preds)

In [ ]:
md.targets.validate_model(ensemble_oof_preds, 'Ensemble Model')

<p style="font-size: 125%; text-align: left; border-radius: 40px 40px;font-weight: bold;">テストデータの予測</p>

In [ ]:
ranked_preds = np.array([rankdata(p) for p in preds])

In [ ]:
ensemble_preds = np.dot(CFG.weights, ranked_preds)

In [ ]:
subm_data = pd.read_csv(CFG.subm_path)
subm_data['prediction'] = ensemble_preds

In [ ]:
subm_data.to_csv('submission.csv', index=False)
display(subm_data.head())